In [ ]:
%%capture
import delfi.distribution as dd
import delfi.utils.io as io
import numpy as np
import pickle

import delfi.inference as infer
import delfi.generator as dg
from delfi.simulator import GaussMixture
import delfi.summarystats as ds
from lfimodels.abc_methods.run_abc import run_smc


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

ptrue = dd.MoG(a=[0.5, 0.5], ms=[np.asarray([0.]), np.asarray([0.])], Ss=[1.0*np.eye(1), 0.01*np.eye(1)])
xx = np.linspace(-1.5,1.5,300).reshape(-1,1)
plt.plot(xx, ptrue.eval(xx, log=False))
plt.show()

In [ ]:
from lfimodels.abc_methods.run_abc import run_smc


def init_g(seed):
    
    m = GaussMixture(dim=1, bimodal=False, noise_cov=[1.0, 0.01], seed=seed)
    p = dd.Uniform(lower=[-10], upper=[10], seed=seed)
    s = ds.Identity()    
    return dg.Default(model=m, prior=p, summary=s, seed=seed)



seeds = range(90, 140)

algos = ['raw', 'AW', 'RA', 'OW']
all_raw, all_AW, all_RA, all_OW = [], [], [], []
all_ = [all_raw, all_AW, all_RA, all_OW]

for seed in seeds:
    for a in range(len(algos)):
                           
        print('\n seed ' + str(seed) + '\n')
            
        g=init_g(seed=seed)
        obs_stats = np.array([[0.]])
        
        kwargs = {'model' : g.model,
                  'prior' : g.prior,
                  'summary' : g.summary, 
                  'obs_stats': obs_stats.copy(),
                  'n_particles' : 5000, 
                  'eps_init' : [2, 0.5, 0.025], 
                  'maxsim' : 1e6,
                  'fn' : None,
                  'seed': seed}
        
        AW     = True if algos[a] in ['AW', 'RA', 'OW'] else False
        regAdj = True if algos[a] in ['RA', 'OW'] else False
        OW     = True if algos[a] in ['OW'] else False
        
        out = run_smc(**kwargs, 
                      AW=AW, 
                      regAdj=regAdj, 
                      OW=OW)
        #plot_res(out)
        all_[a].append(out)

In [ ]:

ess =   np.zeros((len(algos),3,len(seeds)))
nsims = np.zeros((len(algos),3,len(seeds)))
n_particles = np.zeros((len(algos),3,len(seeds)))
ll    = np.zeros((len(algos),3,len(seeds)))

for i in range(len(seeds)):
    for a in range(len(algos)):
        
        res = all_[a][i]

        nsims[a, :, i] = res[4]
        n_particles[a, :, i] = kwargs['n_particles']
        for r in range(3):
            w = np.exp(res[2][r])
            w = w / w.sum()
            ess[a,r,i] = 1./(np.sum(w**2) * w.size)
            ll[a,r,i] = np.sum(ptrue.eval(res[0][r],log=True))


In [ ]:
plt.figure(figsize=(16,5))
for r in range(3):
    plt.subplot(1,3,r+1)
    plt.imshow(ess[:,r,:].T, interpolation='None', aspect='auto')
    plt.colorbar()
    print('\n r = ' + str(r+1) + '\n')
    print(np.mean(ess[:,r,:],axis=1))
    plt.title('round r = ' + str(r+1))
plt.suptitle('ESS')
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
for r in range(3):
    plt.subplot(1,3,r+1)
    plt.imshow(nsims[:,r,].T, interpolation='None', aspect='auto')
    plt.colorbar()
    print('\n r = ' + str(r+1) + '\n')
    print(np.mean(nsims[:,r,:],axis=1))
    plt.title('round r = ' + str(r+1))
plt.suptitle('nsims')
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
for r in range(3):
    plt.subplot(1,3,r+1)
    tmp = nsims[:,r,:].T/(ess[:,r,:]*n_particles[:,r,:]).T
    plt.imshow(tmp, interpolation='None', aspect='auto')
    plt.colorbar()
    print('\n r = ' + str(r+1) + '\n')
    print(np.mean(tmp.T,axis=1))
    plt.title('round r = ' + str(r+1))
plt.suptitle('nsims / ess')
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
for r in range(3):
    plt.subplot(1,3,r+1)
    tmp = ll[:,r,:].T
    plt.imshow(tmp, interpolation='None', aspect='auto')
    plt.colorbar()
    print('\n r = ' + str(r+1) + '\n')
    print(np.mean(tmp.T,axis=1))
    plt.title('round r = ' + str(r+1))
plt.suptitle('log-probability of posterior draw under gt distr.')
plt.show()

In [ ]:
import scipy
from lfimodels.abc_methods.run_abc import set_kernel_bandwidths

for r in range(3):
    plt.figure(figsize=(16,5))

    xx = np.linspace(-3,3,200)
    for a in range(len(algos)):

        
        plt.subplot(1,len(algos),a+1)
        plt.plot(xx, ptrue.eval(xx.reshape(-1,1), log=False), 'g', linewidth=3)
        plt.axis([-3, 3, 0., 2.5])
        plt.title(algos[a])
    
    xx = np.linspace(-3,3,30)
    for i in range(len(seeds)):
        for a in range(len(algos)):

            plt.subplot(1,len(algos),a+1)
            res = all_[a][i]
            plt.plot(xx+np.diff(xx)[0]/2, 
                     np.histogram(res[0][r][:,0], 
                                  weights=np.exp(res[2][r]), 
                                  bins=np.concatenate((xx, xx[-1:])),
                                  normed=True)[0],
                     'k')

        
    plt.ylabel('round #' + str(r+1))
    plt.show()        

In [ ]:
import scipy
from lfimodels.abc_methods.run_abc import set_kernel_bandwidths
plt.figure(figsize=(16,5))

xx = np.linspace(-3,3,300)

for i in range(len(seeds)):
    for a in range(len(algos)):
        
        plt.subplot(1,len(algos),a+1)
        res = all_[a][i]
        bw = set_kernel_bandwidths(data=res[0][-1], 
                                   weights=np.exp(res[2][-1]), 
                                   obs_stats=obs_stats.copy(), 
                                   rule='of_thumb')[0,0]
        #kde = scipy.stats.gaussian_kde(res['posterior'].T, bw)
        #plt.plot(xx,kde.pdf(xx.reshape(1,-1)), 'k')
        
        kde = dd.MoG(a=np.exp(res[2][-1]), 
                     ms=[res[0][-1][ii] for ii in range(res[0][-1].size)],
                     Ss=[  np.atleast_2d(bw**2)  for ii in range(res[0][-1].size)]
                    )
        plt.plot(xx,kde.eval(xx.reshape(-1,1), log=False), 'k')

for a in range(len(algos)):

    plt.subplot(1,len(algos),a+1)
    plt.plot(xx, ptrue.eval(xx.reshape(-1,1), log=False), 'g')
    plt.axis([-3, 3, 0., 2.5])
    plt.title(algos[a])
plt.show()        